In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Albany,42.6001,-73.9662,69.37,67,0,6.85,US,2022-07-26 03:06:32
1,1,Ketchikan,55.3422,-131.6461,66.67,72,20,10.36,US,2022-07-26 03:06:52
2,2,Busselton,-33.6500,115.3333,61.52,80,100,17.81,AU,2022-07-26 03:08:00
3,3,Bambous Virieux,-20.3428,57.7575,67.21,94,75,3.44,MU,2022-07-26 03:08:01
4,4,Cape Town,-33.9258,18.4232,59.11,60,4,3.85,ZA,2022-07-26 03:08:01


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [8]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)

In [6]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [9]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds =  city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)
fig.add_layer(heat_layer)
# Call the figure to plot the data.

fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [14]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,Chitose,42.8194,141.6522,82.09,78,75,11.50,JP,2022-07-26 03:08:02
8,8,Rikitea,-23.1203,-134.9692,75.52,73,57,10.94,PF,2022-07-26 03:08:03
10,10,Bathsheba,13.2167,-59.5167,83.01,94,75,13.80,BB,2022-07-26 03:08:04
14,14,Turayf,31.6725,38.6637,76.82,32,0,6.17,SA,2022-07-26 03:08:06
16,16,Coahuayana,18.7333,-103.6833,77.92,90,86,1.92,MX,2022-07-26 03:03:50
18,18,Marawi,7.9986,124.2928,82.51,63,97,2.64,PH,2022-07-26 03:08:07
21,21,Butaritari,3.0707,172.7902,82.54,70,99,15.05,KI,2022-07-26 03:06:51
25,25,Bubaque,11.2833,-15.8333,78.39,81,85,11.74,GW,2022-07-26 03:08:11
28,28,Mulki,13.1000,74.8000,78.08,83,75,4.61,IN,2022-07-26 03:08:12
29,29,Lorengau,-2.0226,147.2712,84.13,70,73,9.57,PG,2022-07-26 03:06:42


In [15]:
preferred_cities_df.count()

City_ID       195
City          195
Lat           195
Lng           195
Max Temp      195
Humidity      195
Cloudiness    195
Wind Speed    195
Country       195
Date          195
dtype: int64

In [16]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Chitose,JP,82.09,42.8194,141.6522,
8,Rikitea,PF,75.52,-23.1203,-134.9692,
10,Bathsheba,BB,83.01,13.2167,-59.5167,
14,Turayf,SA,76.82,31.6725,38.6637,
16,Coahuayana,MX,77.92,18.7333,-103.6833,
18,Marawi,PH,82.51,7.9986,124.2928,
21,Butaritari,KI,82.54,3.0707,172.7902,
25,Bubaque,GW,78.39,11.2833,-15.8333,
28,Mulki,IN,78.08,13.1000,74.8000,
29,Lorengau,PG,84.13,-2.0226,147.2712,


In [17]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [25]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [26]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))